In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from ydata_profiling import ProfileReport

In [ ]:
pd.set_option('display.max_columns', None) # show all columns in a df

In [ ]:
X_val = pd.read_csv("split_data/val_features.csv")
y_val = pd.read_csv("split_data/val_target.csv")

In [ ]:
df_val = pd.concat([X_val, y_val], axis = 1)

In [ ]:
df_val.shape

In [ ]:
# df_val['row_prop_missing'] = df_val.isna().mean(axis=1) # Not ok to already calculate here - some are missing by design (e.g. sticker)

In [ ]:
df_val.head()

In [ ]:
profile = ProfileReport(df_val)

In [ ]:
profile.to_notebook_iframe()

## Rows with many NAs

In [ ]:
# display(df_val[df_val['row_prop_missing'] > .10])

## Duplicate rows

## Subtype

Observations / remarks:
- Missing values are partly houses and partly appartments -> can be assigned accordingly
- There are synonyms -> can be grouped together
- 'Andere' is never an appartment
- Group infrequent levels together? Reduce dimensionality (but: not really necessary for DT-based approach?)
- Perform clustering a.o. on price to reduce number of levels?

In [ ]:
pd.crosstab(df_val['subtype'], df_val['is_appartment'], dropna=False)

In [ ]:
with open('intermediate_data/map_infrequent_subtypes.pkl', 'rb') as file:
    map_infrequent_subtypes = pickle.load(file)

df_val['subtype_regrouped'] = df_val['subtype'].apply(lambda x: 'Andere' if x in map_infrequent_subtypes else x) # remove this step?

In [ ]:
map_synonyms = {
    'Assistentie-appartement': 'Serviceflat',
    'Villa-landhuis': 'Villa',
    'Moderne villa': 'Villa',
    'Eengezinswoning': 'Woning',    
    # 'Herenwoning': 'Herenhuis',
    # 'Dakappartement': 'Penthouse',
    'Studio met slaaphoek': 'Studio',

    # 'Rijwoning': 'Woning',
    'Gelijkvloers app.': 'Appartement',
    'Uitzonderlijke woning': 'Villa',
    'Herenwoning': 'Villa',
    'Herenhuis': 'Villa',
    'Burgerswoning': 'Woning',
    'Koppelwoning': 'Woning',
    'Duplex': 'Appartement',
    'Triplex': 'Appartement',
    'Bungalow': 'Woning',
    'Hoeve': 'Villa',
    'Fermette': 'Woning',
    'Bel-étage': 'Woning',
    'Hoekwoning': 'Woning',
    'Pastorijwoning': 'Woning',
    'Arbeiderswoning': 'Woning',
    'Loft': 'Loft Penthouse',
    'Dakappartement': 'Appartement',
    'Penthouse': 'Loft Penthouse',
    'Chalet': 'Andere',
    'Cottage': 'Andere',
    'Vakantiewoning': 'Andere'
    
    
}

df_val['subtype_regrouped'] = df_val['subtype_regrouped'].replace(map_synonyms)

df_val['subtype_regrouped'].value_counts()

## Area

Impute area with median value for subtype (regrouped) and province

In [ ]:
df_val['area_missing'] = df_val['area'].isna().astype(int)

In [ ]:
with open('intermediate_data/median_area.pkl', 'rb') as file:
    median_area = pickle.load(file)

In [ ]:
df_val['area_imputed'] = df_val.apply(
    lambda row: median_area.get((row['subtype_regrouped'], row['province']), row['area']) if pd.isna(row['area']) else row['area'],
    axis=1
)

In [ ]:
df_val['area_rel_to_bedrooms'] = df_val['area_imputed'] / (df_val['bedrooms'] + 1)

## Energy value

In [ ]:
# Keep only first letter of energy label, except for 'a+'

df_val['energy_label_regrouped'] = df_val['energy_label'].apply(lambda x: x[0] if isinstance(x, str) and x != 'a+' and x != 'a+' else x)

In [ ]:
pd.crosstab(df_val['energy_label_regrouped'], df_val['new_building'], dropna=False)

For f and g not ok: normally label A corresponds to values 0-100, B to 101-200, etc.

In [ ]:
df_val['energy_value_missing'] = df_val['energy_value'].isna().astype(int)

In [ ]:
with open('intermediate_data/median_energy.pkl', 'rb') as file:
    median_energy = pickle.load(file)

df_val['energy_value_imputed'] = df_val.apply(
    lambda row: median_energy.get((row['energy_label_regrouped'], row['new_building'], row['subtype_regrouped']), row['energy_value']) if pd.isna(row['energy_value']) else row['energy_value'],
    axis=1
)

In [ ]:
# Now all rows except for the ones where the energy label is also missing have an energy value

df_val['energy_value_imputed'].value_counts(dropna = False)

In [ ]:
df_val.groupby(['subtype_regrouped', 'new_building'])['energy_value'].median()

In [ ]:
with open('intermediate_data/median_energy_wo_label.pkl', 'rb') as file:
    median_energy_wo_label = pickle.load(file)

df_val['energy_value_imputed'] = df_val.apply(
    lambda row: median_energy_wo_label.get((row['new_building'], row['subtype_regrouped']), row['energy_value']) if pd.isna(row['energy_value_imputed']) else row['energy_value_imputed'],
    axis=1
)

In [ ]:
# All NAs imputed

df_val['energy_value_imputed'].value_counts(dropna = False)

## Advertiser

High number of categories - apply something similar to Weights Of Evidence (= for classification problems)

Note: some advertisers occur only once and seem to have a person's name (not a real estate agency) -> also informative

Make bins of advertisers based on how often they occur (only once (category 5: person) vs. more (categories 1-4: agency)) and their median price.
Category 5: only occurs once, so no relevant information on 'typical' pricing; this advertiser will (normally) not occur in the test set either 
Catgories 1-4: occurs more than once, categorize based on median price 

In [ ]:
df_val['advertiser'].value_counts(dropna = False).head(50) 

In [ ]:
with open('intermediate_data/median_price_advertiser.pkl', 'rb') as file:
    median_price_advertiser = pickle.load(file)

In [ ]:
df_val = pd.merge(df_val, median_price_advertiser, how='left', on='advertiser')
display(df_val)

In [ ]:
df_val['advertiser_bin'].value_counts(dropna = False)

In [ ]:
df_val['advertiser_bin'].value_counts(dropna = False)

In [ ]:
df_val['advertiser_bin'] = np.where(pd.isna(df_val['advertiser_bin']), 5, df_val['advertiser_bin'])

In [ ]:
df_val['advertiser_bin'].value_counts(dropna = False)

In [ ]:
with open('intermediate_data/median_price_per_advertiser_bin.pkl', 'rb') as file:
    median_price_per_advertiser_bin = pickle.load(file)

In [ ]:
df_val = pd.merge(df_val, median_price_per_advertiser_bin, how='left', on='advertiser_bin')
df_val.head()

## Regional prices - Statbel

In [ ]:
with open('intermediate_data/map_nis_deduplicated.pkl', 'rb') as file:
    df_map_nis_deduplicated = pickle.load(file)

In [ ]:
df_val = pd.merge(df_val, df_map_nis_deduplicated, how='left', left_on='postcode', right_on = 'zip_code')
display(df_val)

In [ ]:
with open('intermediate_data/statbel_prices_selection.pkl', 'rb') as file:
    df_statbel_prices_selection = pickle.load(file)

In [ ]:
df_val = pd.merge(df_val, df_statbel_prices_selection, how='left', left_on=['nis_code', 'is_appartment'], right_on = ['CD_REFNIS', 'F_APPARTMENT'])

In [ ]:
display(df_val)

In [ ]:
# Still quite some missings, whereas profiling below suggests it's a highly relevant feature

df_val['MS_P_50_median'].isna().sum()

In [ ]:
with open('intermediate_data/price_lookup_zipcode.pkl', 'rb') as file:
    price_lookup_zipcode = pickle.load(file)

In [ ]:
df_val["zip_code_first2"] = df_val.zip_code.astype(str).str[:2]

df_val["MS_P_50_median_imputed"] = df_val.apply(
    lambda row: price_lookup_zipcode.get(
        (row["zip_code_first2"], row["is_appartment"]),
        row["MS_P_50_median"]
    ) if pd.isna(row["MS_P_50_median"]) else row["MS_P_50_median"],
    axis=1
)

In [ ]:
with open('intermediate_data/price_lookup_province.pkl', 'rb') as file:
    price_lookup_province = pickle.load(file)

In [ ]:
df_val["zip_code_first1"] = df_val.zip_code.astype(str).str[:1]

df_val["MS_P_50_median_imputed"] = df_val.apply(
    lambda row: price_lookup_province.get(
        (row["zip_code_first1"], row["is_appartment"]),
        row["MS_P_50_median"]
    ) if pd.isna(row["MS_P_50_median_imputed"]) else row["MS_P_50_median_imputed"],
    axis=1
)

In [ ]:
df_val[df_val['MS_P_50_median_imputed'].isna()]

## Regional prices - Price per area 

## Profile again

In [ ]:
# Leave out percentiles 25 and 75 for prices per NIS code because of multicollinearity
# Leave out row count because it's not available when making a single prediction and has low predictive power (it's usually just 1)

df_val_sel = df_val[['bedrooms', 'new_building', 'foto_amount', 'province', 'subtype_regrouped', 'area_missing', 'area_imputed', 'area_rel_to_bedrooms', 'energy_value_missing', 'energy_value_imputed', 'advertiser_count', 'median_price_advertiser_bin', 'MS_TOTAL_TRANSACTIONS', 'MS_P_50_median_imputed', 'price']].drop_duplicates()

## Final variable selection and encoding

In [ ]:
display(df_val_sel)

In [ ]:
# df_val_sel = df_val_sel[df_val_sel['price'] < 500000]

In [ ]:
X_val_preprocessed = df_val_sel.drop('price', axis = 1)
X_val_preprocessed = pd.get_dummies(X_val_preprocessed, columns=['subtype_regrouped', 'province'], drop_first=True)

In [ ]:
display(X_val_preprocessed)

In [ ]:
y_val_preprocessed = df_val_sel['price']

In [ ]:
X_calib_preprocessed, X_val_preprocessed, y_calib_preprocessed, y_val_preprocessed = train_test_split(X_val_preprocessed, y_val_preprocessed, test_size=.5, random_state=42)

In [ ]:
X_calib_preprocessed.shape

In [ ]:
y_calib_preprocessed.shape

In [ ]:
X_val_preprocessed.shape

In [ ]:
y_val_preprocessed.shape

In [ ]:
X_calib_preprocessed.to_csv("split_data/calib_features_preprocessed.csv", index=False)
y_calib_preprocessed.to_csv("split_data/calib_target_preprocessed.csv", index=False)

X_val_preprocessed.to_csv("split_data/val_features_preprocessed.csv", index=False)
y_val_preprocessed.to_csv("split_data/val_target_preprocessed.csv", index=False)